# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [2]:
env = UnityEnvironment(file_name='./Banana_Linux/Banana.x86_64', no_graphics=True)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Instantiate Agent

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))
print()

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)
print()

# examine the state space 
state = env_info.vector_observations[0]
state_size = len(state)
print('States have length:', state_size)
print('States look like:', state)
print()

Number of agents: 1

Number of actions: 4

States have length: 37
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]



In [5]:
from dqn_agent import Agent

agent = Agent(state_size=state_size, action_size=action_size, seed=0)

### 3. Train the Agent

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [6]:
from collections import deque

import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
class EpsScheduler():
    def __init__(self, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
        self.EPS_START = eps_start
        self.EPS_END = eps_end
        self.EPS_DECAY = eps_decay
        
        self.curr_episode = 0

        self.curr_eps = self.EPS_START


    def get_eps(self, i_episode):
        self.curr_eps = max(self.EPS_END, self.curr_eps*(self.EPS_DECAY**(i_episode-self.curr_episode)))
        self.curr_episode = i_episode
        
        return self.curr_eps

In [8]:
def _check_solved(i_episode, scores_window, target=13.0) -> bool:
    print(f'Episode {i_episode}\tAverage Score: {np.mean(scores_window):.2f}', end='\r')
    if i_episode % 100 == 0:
        print(f'Episode {i_episode}\tAverage Score: {np.mean(scores_window):.2f}')
    
    if np.mean(scores_window)>=target:
        print(f'Environment solved in {i_episode-100} episodes!\tAverage Score: {np.mean(scores_window):.2f}')
        return True

    return False

In [ ]:
def dqn(n_episodes=2000, max_t=300, eps_scheduler=EpsScheduler()):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []
    scores_window = deque(maxlen=100)

    for i_episode in range(1, n_episodes+1):
        # Init Score
        score = 0
        
        # Init State
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]

        # Init Eps
        eps = eps_scheduler.get_eps(i_episode)
        
        # Run Episode
        for t in range(max_t):
            # Select Action
            action = agent.act(state, eps)
            
            # Execute Action in Environment
            env_info = env.step(action)[brain_name]
            next_state, reward, done = env_info.vector_observations[0], env_info.rewards[0], env_info.local_done[0]

            # Learn
            agent.step(state, action, reward, next_state, done)
            
            # Update for next Timestep
            state = next_state
            score += reward
            if done:
                break 

        scores_window.append(score)
        scores.append(score)
        
        if _check_solved(i_episode, scores_window):
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')

    return scores


scores = dqn()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Episode 100	Average Score: 0.53
Episode 200	Average Score: 3.21
Episode 300	Average Score: 5.81


When finished, you can close the environment.

In [ ]:
env.close()